In [1]:
from bertopic import BERTopic
import pandas as pd
from tqdm import tqdm

path = './data/'

In [2]:
topic_model = BERTopic.load("{}topic_model_final".format(path))

In [4]:
df = pd.read_parquet('{}phrases-output-final-phrases.parquet'.format(path))
df

,doi,file_name,text,phrases
0,10.1016/j.biortech.2010.01.041,10.1016@j.biortech.2010.01.041.pdf,Geranylation of benzoic acid derivatives by en...,[Geranylation of benzoic acid derivatives by e...
1,10.1590/S0100-40422010000200017,10.1590@S0100-40422010000200017.pdf,"Quim. Nova, Vol. 33, No. 2, 321-323, 2010Artig...","[Quim. Nova, Vol. 33, No. 2, 321-323, 2010Arti..."
2,10.1590/S0103-50532006000500017,10.1590@S0103-50532006000500017.pdf,"J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934, ...","[J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934,..."
3,10.1002/pca.818,10.1002@pca.818.pdf,FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM CR...,[FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM C...
4,10.1002/ps.478,10.1002@ps.478.pdf,Biological activity of astilbin from against ...,[Biological activity of astilbin from against...
...,...,...,...,...
385,10.1016/S0031-9422(00)94671-5,10.1016@S0031-9422(00)94671-5.pdf,"Short Reports 1671 Dichrographe Mark III, R. J...","[Short Reports 1671 Dichrographe Mark III, R. ..."
386,10.1016/S0031-9422(00)80444-6,10.1016@S0031-9422(00)80444-6.pdf,"Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296, ...","[Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296,..."
387,10.1021/np50032a021,10.1021@np50032a021.pdf,Journal of Natural Products Vol. 47. No. 2. pp...,[Journal of Natural Products Vol. 47. No. 2. p...
388,10.1016/S0031-9422(00)83467-6,10.1016@S0031-9422(00)83467-6.pdf,"Phytockmistry, Vol. 23, No. 8, pp. 1677-1679, ...","[Phytockmistry, Vol. 23, No. 8, pp. 1677-1679,..."


In [5]:
topics = []
for phrases_list in tqdm(df.phrases.values):
    phrases_topics = []
    for phrase in phrases_list:
        phrases_topics.append(topic_model.find_topics(phrase)[0])
    topics.append(list(set([item for sublist in phrases_topics for item in sublist])))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [11:00<00:00,  1.69s/it]


In [6]:
from copy import deepcopy

topics_flat = pd.Series([item for sublist in topics for item in sublist])
topics_count = {}
for topic in topics_flat.unique():
    topics_count[topic] = len(topics_flat[topics_flat == topic])/len(topics_flat.unique())
to_remove = [-1, 0]
for key, value in topics_count.items():
    if key == -1 or key == 0:
        continue
    if value > 0.8:
        to_remove.append(key)
print("to remove: {}".format(len(to_remove)))
topics_filtered = deepcopy(topics)
for index, topic_list in enumerate(topics_filtered):
    new_list = []
    for topic in topic_list:
        if topic not in to_remove:
            new_list.append(topic)
    topics_filtered[index] = new_list
print("topics flat original: {}".format(len(topics_flat)))
print("topics flat filtered: {}".format(len([item for sublist in topics_filtered for item in sublist])))


to remove: 58
topics flat original: 17411
topics flat filtered: 2696


In [7]:
df['topics'] = topics_filtered
df

,doi,file_name,text,phrases,topics
0,10.1016/j.biortech.2010.01.041,10.1016@j.biortech.2010.01.041.pdf,Geranylation of benzoic acid derivatives by en...,[Geranylation of benzoic acid derivatives by e...,"[15, 29, 37, 41, 52, 59, 61, 68, 99, 126, 127]"
1,10.1590/S0100-40422010000200017,10.1590@S0100-40422010000200017.pdf,"Quim. Nova, Vol. 33, No. 2, 321-323, 2010Artig...","[Quim. Nova, Vol. 33, No. 2, 321-323, 2010Arti...","[29, 80, 93, 112]"
2,10.1590/S0103-50532006000500017,10.1590@S0103-50532006000500017.pdf,"J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934, ...","[J. Braz. Chem. Soc., Vol. 17, No. 5, 929-934,...","[27, 41, 57, 61, 68, 76, 86]"
3,10.1002/pca.818,10.1002@pca.818.pdf,FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM CR...,[FLAVONOID GLYCOSIDES AND STYRYLPYRONES FROM C...,"[15, 80, 83, 86, 93, 112]"
4,10.1002/ps.478,10.1002@ps.478.pdf,Biological activity of astilbin from against ...,[Biological activity of astilbin from against...,"[129, 135, 18, 19, 21, 37, 38, 61, 77, 93, 105..."
...,...,...,...,...,...
385,10.1016/S0031-9422(00)94671-5,10.1016@S0031-9422(00)94671-5.pdf,"Short Reports 1671 Dichrographe Mark III, R. J...","[Short Reports 1671 Dichrographe Mark III, R. ...","[135, 14, 29, 43, 57, 68, 71, 112]"
386,10.1016/S0031-9422(00)80444-6,10.1016@S0031-9422(00)80444-6.pdf,"Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296, ...","[Phytochemrstry, Vol. 23, No 6, pp. 1X33-1296,...","[14, 80, 84]"
387,10.1021/np50032a021,10.1021@np50032a021.pdf,Journal of Natural Products Vol. 47. No. 2. pp...,[Journal of Natural Products Vol. 47. No. 2. p...,[]
388,10.1016/S0031-9422(00)83467-6,10.1016@S0031-9422(00)83467-6.pdf,"Phytockmistry, Vol. 23, No. 8, pp. 1677-1679, ...","[Phytockmistry, Vol. 23, No. 8, pp. 1677-1679,...","[41, 57, 68, 84, 88, 112]"


In [8]:
import numpy as np
len_list = [len(topic_list) for topic_list in df.topics.values]
df.iloc[len_list.index(np.max(len_list))]

doi                            10.1590/S0100-40422002000700009
file_name                  10.1590@S0100-40422002000700009.pdf
text         Quim. Nova, Vol. 25, No. 6B, 1091-1095, 2002Ar...
phrases      [Quim. Nova, Vol. 25, No. 6B, 1091-1095, 2002A...
topics       [138, 15, 14, 22, 23, 29, 51, 61, 64, 71, 73, ...
Name: 241, dtype: object

In [9]:
df.to_parquet('{}topic-distribution-output.parquet'.format(path))